In [14]:
import uuid
import time
import pickle
import sys
import gym.spaces
import itertools
import numpy as np
import random
import tensorflow                as tf
import tensorflow.contrib.layers as layers
from collections import namedtuple
from collections import Counter
from importlib import reload
from collections import Counter


from mods import multiplayer_tools, TicTacToe,Players, Policy_Gradient, DQN, Actor_Critic


In [15]:
tf.reset_default_graph()

#Main code for running policy gradient

tf.reset_default_graph()

#Define the placeholders
observation_placeholder = tf.placeholder(shape = [None,2 , 3,3], dtype = tf.int32, name ='observe')
adv_n_placeholder = tf.placeholder(shape = [None], dtype = tf.float32, name ='adv_n')
action_placeholder = tf.placeholder(shape = [None], dtype = tf.int32, name ='action')
mask_placeholder = tf.placeholder(shape=[None, 9], dtype = tf.int32, name= 'mask')
target_placeholder = tf.placeholder(shape = [None,], dtype = tf.float32, name = 'target')


#Define the model, action distribution
model = Policy_Gradient.TicTacToe_model(observation_placeholder, "actor", 9)
model_input_s = Policy_Gradient.policy_distribution(model)

critic = Actor_Critic.TicTacToe_model(observation_placeholder, "critic", 1) 



#Define Loss functions *symbolically*
log_prob, entropy = Policy_Gradient.get_log_prob(model, action_placeholder, mask_placeholder)
loss, update_op_actor = Policy_Gradient.loss_and_update_op(log_prob, entropy, adv_n_placeholder, entropy_coeff = 0)
update_op_critic = DQN.symbolic_Q_update(critic, target_placeholder, action_placeholder)

#start a session
sess =tf.Session()
sess.run(tf.global_variables_initializer())

#Defines player, opponent
player = Players.NN_Player(model, model_input_s, sess, observation_placeholder, duplicate=False, deterministic = False)
opponent = Players.Random_Player()





#start an environment
env = TicTacToe.mnk_game()

#Set parameters for update
number_target_updates = 1000
number_gradient_steps = 5
number_updates_per_expert_update = 10


for i in range(number_target_updates):
    replay_buffer = []

    for k in range(number_gradient_steps):
        print("iteration number", k)

        batch_adv_n = []
        iteration_winners = Counter({0:0,1:0,2:0})

        tic = time.time()
        for i in range(number_updates_per_expert_update):
            #Player and Opponent play games, player follows the policy
            #Illegal moves are never played
            paths, batch_winners = multiplayer_tools.batch_rollout(player, opponent, env, max_time_steps=100)

            #The winners are recorded for recordkeeping
            iteration_winners += batch_winners
            
            #Add new rollouts to the replay buffer
            replay_buffer += paths

            for path in paths:
#                print(path['reward'])
                path['reward'] = np.array(path['reward']) - sess.run(critic, feed_dict ={observation_placeholder:path['observation']}).flatten()
#                print(path['reward'])


            

            #The data of the paths are extracted
            adv_n = Policy_Gradient.sum_of_rewards(paths)
            batch_adv_n = batch_adv_n + adv_n
            boards = np.concatenate([path['observation'] for path in paths])
            masks = np.concatenate([path['mask'] for path in paths])
            actions = np.squeeze(np.concatenate([path["action"] for path in paths])).astype(int)

            #The paths data are fed into the model for updating
            sess.run(update_op_actor, feed_dict = {mask_placeholder: masks, adv_n_placeholder: adv_n, observation_placeholder: boards , action_placeholder: actions})


        #Unwind win data:
    #     print(iteration_winners)
        print("mean adv", np.mean(batch_adv_n))
        print("iteration time", time.time() - tic)
    #     print(paths[0])


    #     expert_player = Players.Expert_Player()
    #     _, expert_batch_winners = multiplayer_tools.batch_rollout(player, expert_player, env, max_time_steps=900)
    #     player_loss_percentage_vs_expert = expert_batch_winners[2]*1.0/(expert_batch_winners[0] + expert_batch_winners[1] + expert_batch_winners[2])
    #     print("loss percent vs expert", player_loss_percentage_vs_expert)
        opponent = Players.NN_Player(model, model_input_s, sess, observation_placeholder)
    
    #Collect samples from our replay buffer
    states, actions, next_states, rewards, masks, not_end_of_path = DQN.sample_paths(replay_buffer, batch_size = 100)

    #Compute target values
    target_values = DQN.compute_target_values(player, next_states, masks, not_end_of_path, rewards, verbose=False)

    #Update the network
    player.session.run(update_op_critic, feed_dict= {observation_placeholder : states, action_placeholder : actions, target_placeholder : target_values })
    print("Size of replay buffer:", len(replay_buffer))







iteration number 0
mean adv -0.4808877533544706
iteration time 1.1201491355895996
duplicating session to freeze weights for evaluation...
INFO:tensorflow:Restoring parameters from ./to_duplicate.ckpt
iteration number 1
mean adv -0.5299103288671143
iteration time 1.6376917362213135
duplicating session to freeze weights for evaluation...
INFO:tensorflow:Restoring parameters from ./to_duplicate.ckpt
Destroying NN_Player and Session...
iteration number 2


KeyboardInterrupt: 